<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 3



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [75]:
#using Pkg
#Pkg.add("JuMP")
#Pkg.add("HiGHS")

In [76]:
using JuMP, HiGHS, LinearAlgebra, Printf

## Problema

A Pastesian é uma fábrica de massas familiar que está planejando a produção de lasanhas para os próximos 4 meses. Além do sabor tradicional, ela decidiu lançar um novo sabor de lasanha para esta temporada. A empresa planeja suas operações ao longo de 4 meses. Para o sabor tradicional, a demanda é considerada conhecida com base em dados históricos: 200, 350, 150 e 250 unidades nos meses 1, 2, 3 e 4, respectivamente.

## Parâmetros

In [77]:
# Períodos (mês)
T = 1:4;
# Custo (R$/unidade) de produção em cada mês
cp = [5.5, 7.2, 8.8, 10.9];
# Custo (R$/unidade) de manter as lasanhas em estoque, de um mês para o seguinte
ce = [1.3, 1.95, 2.2];

# Modelo para lasanha original

**Parâmetros**

In [78]:
# Demanda certa nos meses 1–3 (lasanha original)
D_o = [200, 350, 150];
# Lasanhas originais (unidade) em estoque inicialmente
e0_o = 50;

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_o = [220, 250, 300]
# Penalidade por demanda não atendida no mês 4
pp_o = 5.045

5.045

**Modelo**

Este código constrói o modelo base contendo apenas os elementos referentes à lasanha original. Ele representa a parte fixa e comum aos dois problemas, tanto ao modelo determinístico quanto ao estocástico da lasanha nova, e, por isso, será reutilizado posteriormente na formulação de ambos.

Observe que a função objetivo ainda não é definida, pois ela difere entre os dois modelos finais. Aqui, o foco está apenas nas restrições estruturais da lasanha original. Em particular, consideramos:

- O estoque inicial do produto original: `e_o[1] = e0_o`;
- O balanço de estoque ao longo dos três primeiros meses: `e_o[t+1] = e_o[t] + l_o[t] - D_orig[t]` para os meses `t = 1,2,3`;
- A presença de incerteza na demanda do 4° mês, tratada por meio de variáveis específicas para cada cenário, já que não existe um único valor determinístico para esse período;
- A restrição de atendimento da demanda estocástica no mês 4, permitindo falta apenas através da variável de déficit `dp_o[c]`: `dp_o[c] >= D4_o[c] - (l4_o[c] + e_o[4])`;
- A restrição que garante que a produção no mês 4 seja única e idêntica para todos os cenários: `sum(l4_o) == length(C_o) * l4_o[c]`;



In [79]:
function build_base_original(model, T, D_o, e0_o, C, D4_o)
    ## ============================================================
    ## Variáveis
    ## ============================================================
    @variable(model, l_o[T] >= 0)       # produção da lasanha original
    @variable(model, e_o[T] >= 0)       # estoque da lasanha original
    @variable(model, l4_o[C] >= 0)      # produção no mês 4 por cenário
    @variable(model, dp_o[C] >= 0)      # demanda não atendida no mês 4 por cenário

    ## ============================================================
    ## Restrições (meses 1,2,3)
    ## ============================================================

    # Estoque inicial
    @constraint(model, e_o[1] == e0_o)

    # Balanço meses 1, 2, 3
    @constraint(model, BAL_o[t in T[1:end-1]],
        e_o[t+1] == e_o[t] + l_o[t] - D_o[t]
    )

    ## ============================================================
    ## Restrições (mês 4)
    ## ============================================================

    # Atender as demandas incertas no mês 4
    @constraint(model, DEM_o[c in C],
        dp_o[c] >= D4_o[c] - (l4_o[c] + e_o[4])
    )

    # A produção no mês 4 deve ser a mesma em todos os cenários
    @constraint(model, NA_o[c in C],
        sum(l4_o) == length(C) * l4_o[c]
    )

    return model, l_o, e_o, l4_o, dp_o
end

build_base_original (generic function with 3 methods)

# Modelo determinístico para lasanha nova

**Parâmetros**

In [80]:
# Demanda certa nos meses 1–3 (lasanha nova)
D_n = [30, 70, 140];

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_n = [200, 240, 300];
# Probabilidades
P4_n = [0.3, 0.5, 0.2];

# Capacidade total de estoque em cada mês (para as duas lasanhas somadas)
E = [200, 220, 230, 250]

4-element Vector{Int64}:
 200
 220
 230
 250

In [81]:
# Demanda média esperada da lasanha nova no mês 4
d4_mean_n = dot(D4_n, P4_n);
D_n_det = vcat(D_n, d4_mean_n);

**Modelo**

- Adicionar capacidade de estoque
([estoque lasanha antiga no mes 1] + [estoque lasanha nova no mes 1] <= 200 ...)

In [82]:
# Capacidade de estoque
@constraint(model, CE[t = T], e[t] <= 200);

LoadError: UndefVarError: `e` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [83]:
model_det = Model(HiGHS.Optimizer);
model_det, l_o, e_o = build_base_original(model_det, T, D_o, e0_o, C_o, D4_o);

In [84]:
@variable(model_det, l_n[T] >= 0)   # produção da lasanha nova
@variable(model_det, e_n[T] >= 0)   # estoque da lasanha nova

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{VariableRef}:
 e_n[1]
 e_n[2]
 e_n[3]
 e_n[4]

In [85]:
# Estoque inicial da lasanha nova é zero
@constraint(model_det, e_n[1] == 0)

# Balanço de estoque da lasanha nova (meses 1,2,3)
@constraint(model_det, BAL_n[t in T[1:end-1]],
    e_n[t+1] == e_n[t] + l_n[t] - D_n_det[t]
)

# Atendimento à demanda da lasanha nova (de fato só precisa em t=4,
# mas deixamos para todos os meses por simetria didática)
@constraint(model_det, AD_n[t in T],
    l_n[t] + e_n[t] >= D_n_det[t]
)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 AD_n[1] : l_n[1] + e_n[1] ≥ 30
 AD_n[2] : l_n[2] + e_n[2] ≥ 70
 AD_n[3] : l_n[3] + e_n[3] ≥ 140
 AD_n[4] : l_n[4] + e_n[4] ≥ 240

In [86]:
# ============================
# Capacidade de estoque total (original + nova)
# ============================
@constraint(model_det, CE[t in T],
    e_o[t] + e_n[t] <= E[t]
)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 CE[1] : e_o[1] + e_n[1] ≤ 200
 CE[2] : e_o[2] + e_n[2] ≤ 220
 CE[3] : e_o[3] + e_n[3] ≤ 230
 CE[4] : e_o[4] + e_n[4] ≤ 250

In [87]:
# Custo de produção + custo de estoque das DUAS lasanhas
@objective(model_det, Min,
    # Produção (original + nova)
    dot(cp, l_o) + dot(cp, l_n) +
    # Custo de estoque (mês 1→2, 2→3, 3→4) para original + nova
    dot(ce, e_o[2:end] .+ e_n[2:end])
)

5.5 l_o[1] + 7.2 l_o[2] + 8.8 l_o[3] + 10.9 l_o[4] + 5.5 l_n[1] + 7.2 l_n[2] + 8.8 l_n[3] + 10.9 l_n[4] + 1.3 e_o[2] + 1.3 e_n[2] + 1.95 e_o[3] + 1.95 e_n[3] + 2.2 e_o[4] + 2.2 e_n[4]

In [88]:

optimize!(model_det)

println("Status: ", termination_status(model_det))
println("FO ótima = ", objective_value(model_det))

println("\nProdução lasanha original (l_o): ", value.(l_o))
println("Estoque lasanha original (e_o):   ", value.(e_o))

println("\nProdução lasanha nova (l_n):      ", value.(l_n))
println("Estoque lasanha nova (e_n):        ", value.(e_n))

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 22 rows; 22 cols; 54 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 2e+00]
  Cost    [1e+00, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [3e+01, 4e+02]
Presolving model
13 rows, 14 cols, 33 nonzeros  0s
Dependent equations search running on 4 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
10 rows, 11 cols, 24 nonzeros  0s
Presolve reductions: rows 10(-12); columns 11(-11); nonzeros 24(-30) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     9.9000000000e+02 Pr: 7(1160) 0s
          9     9.0940000000e+03 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 9
Objective value     :  9.0940000000e+03
P-D objective error :  0.0000000000e+00
Hi

# Modelo estocástico para lasanha nova

In [89]:
model_estocastico = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none